In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.decomposition import PCA
from feature_engine.outliers import Winsorizer
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.impute import KNNImputer

missing values -- drop, impute
persistent outliers -- drop, don't drop
log transform y -- yes or no
reduce y range -- yes or no
use transformed nutrients -- yes or no
PCA -- yes or no
Winsorize X -- yes or no
include chronic conditions -- yes or no
include alcohol and tobacco -- yes or no
transformed alcohol -- yes or no
transformed physical activity -- yes or no

Want to save model number, r2, adjusted r2, training mae, test mae, training rmse, test rmse


To get an idea of what to do for the model, I'm training ten models picked randomly
1.  Drop missing values, drop persistent outliers, log transform y, use transformed nutrients, PCA, no winsorization, no chronic conditions, include alcohol and tobacco, don't use transformed alcohol, use transformed physical activity
2.  Drop missing values, don't drop outliers, don't log transform y, reduce y range, use transformed nutrients, no PCA, winsorize x values, include chronic conditions, dont include alcohol and tobacco, transformed physical activity.
3.  drop missing values, drop outliers, don't log transform, reduce y range, use transformed nutrients, use PCA, no winsorizatiion, no chronic conditions, include alcohol and tobacco, include transformed alcohol, don't transform physical activity.